In [ ]:
!pip install flask pyngrok pandas folium


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 681.2/681.2 kB 7.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-6.0.0-py3-none-any.whl size=19866 sha256=4304758c9337a209373e200b4ae9be2f306bdbf896e4d5b0b6e50ebca312181b
  Stored in directory: /root/.cache/pip/wheels/5c/42/78/0c3d438d7f5730451a25f7ac6cbf4391759d22a67576ed7c2c
Successfully built pyngrok


In [ ]:
!curl -s https://ngrok-agent.s3.amazonaws.com/ngrok.asc | sudo tee /etc/apt/trusted.gpg.d/ngrok.asc >/dev/null && echo "deb https://ngrok-agent.s3.amazonaws.com buster main" | sudo tee /etc/apt/sources.list.d/ngrok.list && sudo apt update && sudo apt install ngrok

deb https://ngrok-agent.s3.amazonaws.com buster main
Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:3 https://ngrok-agent.s3.amazonaws.com buster InRelease [20.3 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:7 https://ngrok-agent.s3.amazonaws.com buster/main amd64 Packages [3,568 B]
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,333 kB]
Get:10 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease [18.1 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,398 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,678 kB]


In [ ]:
!ngrok authtoken "2T8yAOjN4tt3GhpNVoBSb1Pnv5G_2ZDYHarumrMgH8FesLg59"

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
!unzip templates.zip

Archive:  templates.zip
  inflating: templates/index.html    


In [ ]:
from flask import Flask, render_template, request
import pandas as pd
import folium
from folium.plugins import MarkerCluster
from pyngrok import ngrok

# Create a Flask app
app = Flask(__name__)

# Code 1: Data Loading and Preprocessing
# Load the Excel file into dataframes
excel_file_path = '/content/finaloutput.xlsx'  # Replace with the actual path to your Excel file
sheet_name1 = 'Sheet1'  # Replace with the name of the first sheet
sheet_name2 = 'Sheet2'  # Replace with the name of the second sheet

df1 = pd.read_excel(excel_file_path, sheet_name=sheet_name1)
df2 = pd.read_excel(excel_file_path, sheet_name=sheet_name2)

# Stack the dataframes vertically
finaloutputdata = pd.concat([df1, df2], ignore_index=True)

# Code 2: Filtering Data Based on Time Interval
def filter_data(start_time_str, end_time_str):
    data = finaloutputdata
    df = pd.DataFrame(data)
    df['arrival_time'] = pd.to_datetime(df['arrival_time'])

    # Convert start and end times to pandas Timestamp objects
    start_time = pd.to_datetime(start_time_str, format='%H:%M').time()
    end_time = pd.to_datetime(end_time_str, format='%H:%M').time()

    # Filter the DataFrame based on the time interval
    mask = ((df['arrival_time'].dt.time >= start_time) & (df['arrival_time'].dt.time <= end_time)) | \
           (df['arrival_time'].dt.time == end_time) | \
           (df['arrival_time'].dt.time == start_time)

    filtered_df = df[mask]
    return filtered_df

# Code 3: Creating a Congestion Map
def create_congestion_map(filtered_df):
    stoppage = filtered_df['Stoppage_Id'].value_counts()
    stopcongestion = pd.DataFrame({'Stoppage_Id': stoppage.index, 'Count': stoppage.values})

    stopdata = pd.read_excel('/content/AVL_Stop.xlsx')
    # Creating the map of Delhi by taking the mean latitude and longitude
    m = folium.Map(location=[stopdata['Stop_Latitute'].mean(), stopdata['Stop_Longitude'].mean()], zoom_start=20)

    marker_cluster = MarkerCluster().add_to(m)

    congestion_colors = {
        1: 'green',  # Low congestion
        2: 'yellow',  # Medium congestion
        3: 'orange',  # High congestion
        4: 'red'  # Very high congestion
    }

    for idx, row in stopcongestion.iterrows():
        stop_id = row['Stoppage_Id']
        count = row['Count']

        stop_row = stopdata[stopdata['Stoppage_Id'] == stop_id].iloc[0]
        lat = stop_row['Stop_Latitute']
        lon = stop_row['Stop_Longitude']

        # Default to low congestion
        if count >= 50:
            congestion_level = 4  # Very high congestion
        elif count >= 30:
            congestion_level = 3  # High congestion
        elif count >= 10:
            congestion_level = 2  # Medium congestion
        else:
            congestion_level = 1  # Low congestion

        # Creating a marker with colored icon and pop-up tooltip
        marker = folium.Marker(
            location=[lat, lon],
            icon=folium.Icon(color=congestion_colors[congestion_level], icon='bus', prefix='fa'),
            tooltip=f'Stop ID: {stop_id}<br> Congestion Count: {count}',
        )

        # Adjusting the size and opacity of the marker based on the congestion count
        marker.radius = count * 2
        marker.opacity = min(0.5 + count / 20, 1.0)

        marker.add_to(marker_cluster)

    return m

# Flask route to render the home page with the form
@app.route('/')
def home():
    return render_template('index.html')

# Flask route to handle the form submission
@app.route('/congestion', methods=['POST'])
def congestion_map():
    # Get the start and end times from the form
    start_time_str = request.form.get('start_time')
    end_time_str = request.form.get('end_time')

    filtered_df = filter_data(start_time_str, end_time_str)
    m = create_congestion_map(filtered_df)

    return m._repr_html_()

# Keep the app running until you stop the execution
if __name__ == "__main__":
    # Start ngrok and create a tunnel
    ngrok.set_auth_token("2T8yAOjN4tt3GhpNVoBSb1Pnv5G_2ZDYHarumrMgH8FesLg59")  # Replace "your_auth_token" with your ngrok auth token
    public_url = ngrok.connect(addr="5000", proto="http")
    print("Public URL:", public_url)

    try:
        app.run(port=5000)
    except KeyboardInterrupt:
        pass


Public URL: NgrokTunnel: "https://4e8c-34-125-153-77.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [31/Jul/2023 05:50:39] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [31/Jul/2023 05:50:39] "GET /static/background.mp4 HTTP/1.1" 206 -
INFO:werkzeug:127.0.0.1 - - [31/Jul/2023 05:50:40] "GET /static/background.mp4 HTTP/1.1" 206 -
INFO:werkzeug:127.0.0.1 - - [31/Jul/2023 05:50:41] "GET /static/background.mp4 HTTP/1.1" 206 -
INFO:werkzeug:127.0.0.1 - - [31/Jul/2023 05:50:41] "GET /static/background.mp4 HTTP/1.1" 206 -
INFO:werkzeug:127.0.0.1 - - [31/Jul/2023 05:50:45] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [31/Jul/2023 05:50:49] "POST / HTTP/1.1" 405 -
INFO:werkzeug:127.0.0.1 - - [31/Jul/2023 05:51:03] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [31/Jul/2023 05:51:03] "GET /static/background.mp4 HTTP/1.1" 206 -
INFO:werkzeug

In [ ]:
!unzip d.zip

Archive:  d.zip
  inflating: d/New Text Document.txt  


In [ ]:
!unzip static.zip

Archive:  static.zip
  inflating: static/vadim-sherbakov-NQSWvyVRIJk-unsplash.jpg  


Public URL: NgrokTunnel: "https://8834-34-125-153-77.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [31/Jul/2023 05:27:03] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [31/Jul/2023 05:27:04] "GET /static/background.mp4 HTTP/1.1" 206 -
INFO:werkzeug:127.0.0.1 - - [31/Jul/2023 05:27:04] "GET /static/background.mp4 HTTP/1.1" 206 -
INFO:werkzeug:127.0.0.1 - - [31/Jul/2023 05:27:05] "GET /static/background.mp4 HTTP/1.1" 206 -
INFO:werkzeug:127.0.0.1 - - [31/Jul/2023 05:27:06] "GET /static/background.mp4 HTTP/1.1" 206 -
INFO:werkzeug:127.0.0.1 - - [31/Jul/2023 05:27:09] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [31/Jul/2023 05:27:29] "POST / HTTP/1.1" 200 -


In [ ]:
!unzip k.zip

Archive:  k.zip
  inflating: k/New Text Document.txt  
